# Tool Agent

In [6]:
!pip install python-dotenv langchain-community wikipedia


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [8]:
from app.llm.openai.service import OpenAIService
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

llm = OpenAIService(api_key=api_key)

In [9]:
from app.agents.react.agent import ReActAgentFactory
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from app.tools.langchain.wrapper import LangChainToolWrapper

api_wrapper = WikipediaAPIWrapper(
    top_k_results=3,
    doc_content_chars_max=1500,
    load_all_available_meta=True
)

tools = [
    LangChainToolWrapper.create(
        WikipediaQueryRun(api_wrapper=api_wrapper),
        exclude_input_from_scratch_pad=False,
        exclude_output_from_scratch_pad=True
    )
]
react_agent = ReActAgentFactory.start(llm=llm, tools=tools, step_limit=8, clear_scratch_pad_after_answer=True)

In [10]:
import json
from app.core.messages import Query

future = react_agent.ask(Query(goal="What year did the iPhone come out?"))
response = future.get(timeout=120)
print(f"Steps: {json.dumps([step.model_dump() for step in response.metadata['steps']], indent=2)}")
print(f"Total tokens: {sum(step.token_usage.total_tokens for step in response.metadata['steps'])}")
print(response.final_output)

future = react_agent.ask(Query(goal="Can you give me a list of all the models?"))
response = future.get(timeout=120)
print(f"Steps: {json.dumps([step.model_dump() for step in response.metadata['steps']], indent=2)}")
print(f"Total tokens: {sum(step.token_usage.total_tokens for step in response.metadata['steps'])}")
print(response.final_output)

react_agent.stop()

Steps: [
  {
    "state_name": "States.THOUGHT",
    "next_state": "States.ACTION",
    "prompt": "You're a helpful assistant called ReActAgent. You solve problems by breaking them down into multiple steps, thinking on those steps, acting on them, and observing. Given the problem, you will use your tools to solve it in as few steps as possible.\n\nCreate a thought based on the message.\n\nThought instructions:\n- Provides exact details on the task to best answer the message, do not forget important details.\n- You have access to tools that can help you answer the message. Always try to use a tool.\n-- In your thought, recommend a tool that will help.\n-- If the problem can be solved using the tool, always use the tool.\n-- If the problem cannot be solved or if you are unsure, give your answer with why.\n- If you keep running into issues with your tools, give your answer with the problems you're running into.\n- If the message doesn't require any tool, just give your answer.\n- Don't re

True